-----
<h1><font color="#f37626">[Experiment]</font> Keras-Regression 예제</h1>

- 보다 상세한 Accuinsight 파이썬 패키지 사용법은 [Accuinsight 안내 홈페이지](https://accuinsight.cloudz.co.kr/#/intro) 또는 [Accuinsight Youtube 채널](https://www.youtube.com/channel/UChFs-FAVxgG4C00h8C1MqoA)을 참조하시기 바랍니다.
- Accuinsight 패키지를 사용한 분석 코드는 [Accuinsight-github](https://github.com/AccuInsight/accuinsight_Lifecycle_example)에서 조회 가능합니다.

###  # 보스턴 주택 가격 예측
----

### 1. Import packages

In [1]:
from Accuinsight.Lifecycle.tensorflow import accuinsight

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import pandas as pd
import numpy as np

2021-12-01 16:15:51.732109: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
accu = accuinsight()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

### 2. Data load and split

In [3]:
accu.set_storage(file_path = '../data/boston_data.csv', target='MEDV')

In [4]:
boston = pd.read_csv('../data/boston_data.csv')
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [5]:
from sklearn.model_selection import train_test_split

boston_train, boston_valid = train_test_split(boston, test_size = 0.4)
boston_valid, boston_test = train_test_split(boston_valid, test_size = 0.5)

y_train = boston_train.loc[:, 'MEDV']
y_valid = boston_valid.loc[:, 'MEDV']
y_test = boston_test.loc[:, 'MEDV']

X_train = boston_train.drop(['MEDV'], axis = 1)
X_valid = boston_valid.drop(['MEDV'], axis = 1)
X_test = boston_test.drop(['MEDV'], axis = 1)

X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 54 to 117
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     303 non-null    float64
 1   ZN       303 non-null    float64
 2   INDUS    303 non-null    float64
 3   CHAS     303 non-null    int64  
 4   NOX      303 non-null    float64
 5   RM       303 non-null    float64
 6   AGE      303 non-null    float64
 7   DIS      303 non-null    float64
 8   RAD      303 non-null    int64  
 9   TAX      303 non-null    int64  
 10  PTRATIO  303 non-null    float64
 11  B        303 non-null    float64
 12  LSTAT    303 non-null    float64
dtypes: float64(10), int64(3)
memory usage: 33.1 KB


### 3. Preprocessing

In [6]:
train_stats = X_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
CRIM,303.0,3.190529,7.811638,0.00906,0.07891,0.26363,2.85187,88.9762
ZN,303.0,11.574257,23.166371,0.00000,0.00000,0.00000,12.50000,90.0000
INDUS,303.0,10.756238,6.730806,0.74000,5.13000,8.14000,18.10000,27.7400
CHAS,303.0,0.072607,0.259920,0.00000,0.00000,0.00000,0.00000,1.0000
NOX,303.0,0.550639,0.112969,0.38500,0.44900,0.52400,0.62400,0.8710
RM,303.0,6.272099,0.670044,3.56100,5.87850,6.21600,6.62050,8.7250
AGE,303.0,68.076238,27.932031,6.00000,43.55000,76.90000,93.90000,100.0000
DIS,303.0,3.940754,2.173462,1.12960,2.16475,3.42420,5.40070,12.1265
RAD,303.0,9.227723,8.526408,1.00000,4.00000,5.00000,8.00000,24.0000
TAX,303.0,401.917492,165.570266,187.00000,282.50000,329.00000,666.00000,711.0000


In [7]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_train = norm(X_train)
normed_valid = norm(X_valid)
normed_test = norm(X_test)

### 4. Build model

> __autoDL에서 최적화할 Hyperparameter 지정__

In [8]:
learning_rate = 0.01
num_nodes = 100

def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(X_train.keys())]),
        layers.Dense(num_nodes, activation='relu'),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(learning_rate)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

model = build_model()

2021-12-01 16:16:36.749917: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-12-01 16:16:36.749956: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: UNKNOWN ERROR (-1)
2021-12-01 16:16:36.749982: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ws-343-7ffd47d6f9-jv86q): /proc/driver/nvidia/version does not exist
2021-12-01 16:16:36.750235: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-01 16:16:36.759646: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2499995000 Hz
2021-12-01 16:16:36.760513: I tensorflow/compiler/xla/se

### 5. Run model with experiment
- `autolog()`에 현재 학습할 모델에 관한 간단한 tag를 입력할 수 있습니다.
- `autolog()`는 반드시 모델 학습(model.fit()) __이전에__ 호출이 되어야 합니다.

> __모델 학습이 완료되면 `autolog()`는 자동으로 해제됩니다.__  
따라서 모델 학습 이력을 추가하고자 할 경우, 다시 한 번 `autolog()` 호출 후 모델 훈련을 진행해야 합니다.

#### [optional] 1-(1) 메시지 설정 

message를 푸시하는 방법은 2가지 입니다.
1. 모델 학습 완료시 메시지 푸시  
    - `send_message(message = 'your_message')`  
    
    
2. 학습에 사용되는 metric이 일정 thresholds가 넘은 경우에만 메시지 푸시
    - `send_message(thresholds = 0.5)`

In [9]:
accu.send_message('[Accuinsight+] Experiment 실행 완료')

#### [optional] 1-(2) alarm method 설정
- web push는 기본 method이며, message가 있을 경우 자동으로 alart 됩니다.
- slack: slack channel의 hook url을 입력합니다.
- mail: mail address를 입력합니다.

In [ ]:
# accu.set_slack(hook_url='hook_url')

- autolog()를 사용하기 위해서는 `validation_data`를 반드시 지정해주어야 합니다.

In [10]:
# model_monitor=False
# accu.autolog('boston-house-pricing', best_weights=True)

# model_monitor=True
accu.autolog('boston-house-pricing', best_weights=True, model_monitor=True)

In [ ]:
model.fit(normed_train, y_train,
          epochs=10,
          validation_data=(normed_valid, y_valid))

Using autolog(best_weights=True, model_monitor=True


Epoch 1/10
10/10 [==============================] - 0s 22ms/step - loss: 160.1514 - mae: 9.1860 - mse: 160.1514 - val_loss: 32.4931 - val_mae: 4.5414 - val_mse: 32.4931
Epoch 2/10
10/10 [==============================] - 0s 8ms/step - loss: 32.3030 - mae: 4.1052 - mse: 32.3030 - val_loss: 30.4891 - val_mae: 4.3349 - val_mse: 30.4891
Epoch 3/10
10/10 [==============================] - 0s 8ms/step - loss: 22.3021 - mae: 3.2029 - mse: 22.3021 - val_loss: 17.8396 - val_mae: 3.1815 - val_mse: 17.8396
Epoch 4/10
10/10 [==============================] - 0s 8ms/step - loss: 24.0173 - mae: 3.6366 - mse: 24.0173 - val_loss: 23.4239 - val_mae: 3.6487 - val_mse: 23.4239
Epoch 5/10
10/10 [==============================] - 0s 8ms/step - loss: 20.6573 - mae: 3.3858 - mse: 20.6573 - val_loss: 22.2114 - val_mae: 3.7500 - val_mse: 22.2114
Epoch 6/10
10/10 [==============================] - 0s 8ms/step - loss: 16.8619 - mae: 3.0287 - mse: 16.8619 - val_loss: 14.9876 - val_mae: 3.1177 - val_mse: 14.9876
E

From /opt/conda/lib/python3.7/site-packages/shap/explainers/_deep/deep_tf.py:239: set_learning_phase (from tensorflow.python.keras.backend) is deprecated and will be removed after 2020-10-11.
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.



Using epoch 00008 with val_mae: 3.06301


### 6. Load saved model
- `autolog()`를 사용하여 모델의 학습 이력을 Lifecycle에 기록할 경우, 자동으로 훈련 도중 가장 좋은 metric을 기록한 epoch에서의 모델 가중치가 저장됩니다.
- 따라서 저장된 모델을 불러와 공동 작업자들과 모델을 공유하거나, 모델 재학습을 수행할 수 있습니다.

    1. Accuinsight+ workspace list 혹은 해당 모델의 상세화면으로 접속하여 _Experiment_ 중 불러오고자 하는 모델의 __Run name__을 복사합니다.
    2. ___utils___에서 `load_model()` 함수를 호출하여 모델을 불러올 수 있습니다.

In [17]:
from Accuinsight.Lifecycle.utils import load_model

saved_model = load_model('tf.keras-35FE0DD96251456AA9FC8D6E57EF2AE8_151')

In [18]:
saved_model.get_weights()[0]

array([[-1.01617903e-01, -9.09344032e-02,  2.77260840e-02,
        -2.75677919e-01, -7.09393322e-02,  3.47526511e-05,
        -1.61453784e-01, -5.41245155e-02, -1.76748097e-01,
        -1.82703182e-01,  2.63570454e-02, -1.14701502e-01,
        -1.30410001e-01, -2.45125726e-01, -3.37876320e-01,
        -6.49221241e-02, -2.94812083e-01, -1.24674864e-01,
         1.12674329e-02, -7.34634325e-02,  4.82871644e-02,
         3.12486887e-02, -2.72160172e-02, -2.09686950e-01,
         1.41523197e-01, -1.72494873e-01, -2.81550944e-01,
        -2.03805387e-01, -2.16628984e-01, -7.90450945e-02,
        -2.13329971e-01,  2.49425508e-02, -5.08492664e-02,
         2.65738606e-01,  1.10074729e-01, -1.68908745e-01,
        -3.39695007e-01, -1.42188221e-01,  4.09988835e-02,
        -9.98531580e-02, -3.50689217e-02,  5.94261624e-02,
         6.45288303e-02,  3.78156360e-03,  1.19989425e-01,
        -4.45049815e-03,  5.54582998e-02, -1.21642955e-01,
        -1.34948984e-01, -2.33982548e-01, -1.87384516e-0

-------
### AutoDL에 사용할 데이터 npy 형식으로 저장(AutoDL 사용 시, 사용)
- 반드시 `filestorage`에 저장해야 함

In [19]:
X_train.to_csv('filestorage/X_train.csv')
y_train.to_csv('filestorage/y_train.csv')
X_valid.to_csv('filestorage/X_valid.csv')
y_valid.to_csv('filestorage/y_valid.csv')
X_test.to_csv('filestorage/X_test.csv')
y_test.to_csv('filestorage/y_test.csv')